<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/agent_mistral_medical_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistralai colab-env -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 12.6 MB/s eta 0:00:00


In [2]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Lab Result Analyzer agent's response format
class LabAnalysisResult(BaseModel):
    analysis_summary: str
    recommendations: str
    flagged_values: dict

print("Creating AI agents for Medical Domain...")

# --- Agent Definitions (Medical Domain) ---

# 1. Patient Intake Agent (Refactored from Flight Planning Agent)
patient_intake_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for registering new patients and collecting initial demographic and medical history.",
    name="patient-intake-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "register_patient",
                "description": "Register a new patient with their name, date of birth, and contact info.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_name": {"type": "string", "description": "Full name of the patient."},
                        "dob": {"type": "string", "description": "Date of birth in YYYY-MM-DD format."},
                        "contact_number": {"type": "string", "description": "Patient's contact phone number."}
                    },
                    "required": ["patient_name", "dob", "contact_number"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "retrieve_patient_record",
                "description": "Retrieve an existing patient's basic medical record using their patient ID.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Unique patient identifier."}
                    },
                    "required": ["patient_id"]
                }
            }
        }
    ]
)
print(f"Patient Intake Agent '{patient_intake_agent.name}' created with ID: {patient_intake_agent.id}")

# 2. Medical Research Agent (Refactored from Web Search Agent)
medical_research_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for searching medical literature, clinical guidelines, and research papers.",
    name="medical-research-agent",
    tools=[
        {"type": "web_search"}, # Still uses web search, but for medical context.
        {
            "type": "function",
            "function": {
                "name": "search_medical_literature",
                "description": "Search for medical articles or research papers on a specific topic or condition.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "topic": {"type": "string", "description": "The medical topic or condition to search for."}
                    },
                    "required": ["topic"]
                }
            }
        }
    ]
)
print(f"Medical Research Agent '{medical_research_agent.name}' created with ID: {medical_research_agent.id}")

# 3. Lab Result Analyzer Agent (Refactored from Calculator Agent)
lab_result_analyzer_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="lab-result-analyzer-agent",
    description="Agent used to analyze lab results, highlight anomalies, and provide a summary.",
    instructions="Analyze the provided lab results, identify any values outside normal ranges, and summarize findings.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "lab_analysis_result",
                "schema": LabAnalysisResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_lab_results",
                "description": "Analyze a set of lab results (e.g., blood test) and compare them against normal ranges.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "results": {"type": "string", "description": "A JSON string of lab results, e.g., '{\"glucose\": 120, \"cholesterol\": 250}'."}
                    },
                    "required": ["results"]
                }
            }
        }
    ]
)
print(f"Lab Result Analyzer Agent '{lab_result_analyzer_agent.name}' created with ID: {lab_result_analyzer_agent.id}")

# 4. Medication Management Agent (Refactored from Finance Agent / ECB Interest Rate Agent)
medication_management_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking drug interactions, recommended dosages, and medication information.",
    name="medication-management-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_drug_interaction",
                "description": "Check for potential interactions between two or more medications.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "medication_names": {"type": "array", "items": {"type": "string"}, "description": "List of medication names to check for interactions."}
                    },
                    "required": ["medication_names"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_medication_dosage",
                "description": "Retrieve recommended dosage information for a specific medication.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "medication_name": {"type": "string", "description": "Name of the medication."}
                    },
                    "required": ["medication_name"]
                }
            }
        }
    ]
)
print(f"Medication Management Agent '{medication_management_agent.name}' created with ID: {medication_management_agent.id}")


# 5. Health Monitor Agent (Refactored from Weather Agent)
health_monitor_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for monitoring patient vital signs and symptoms, and identifying potential health alerts.",
    name="health-monitor-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "monitor_vitals",
                "description": "Record and evaluate current vital signs (e.g., heart rate, blood pressure, temperature).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Unique patient identifier."},
                        "vitals_data": {"type": "string", "description": "JSON string of vital signs, e.g., '{\"heart_rate\": 75, \"bp_systolic\": 120}'."}
                    },
                    "required": ["patient_id", "vitals_data"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_symptom",
                "description": "Log a patient's reported symptom with its description and severity.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Unique patient identifier."},
                        "symptom_description": {"type": "string", "description": "Description of the symptom."},
                        "severity": {"type": "string", "description": "Severity of the symptom (e.g., 'mild', 'moderate', 'severe')."}
                    },
                    "required": ["patient_id", "symptom_description", "severity"]
                }
            }
        }
    ]
)
print(f"Health Monitor Agent '{health_monitor_agent.name}' created with ID: {health_monitor_agent.id}")


# 6. Medical Equipment Agent (Refactored from Maintenance Agent)
medical_equipment_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing medical equipment status, scheduling calibration, and reporting malfunctions.",
    name="medical-equipment-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_equipment_status",
                "description": "Check the operational status and next calibration date for a specific medical equipment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Unique identifier for the medical equipment (e.g., 'MRI-001')."}
                    },
                    "required": ["equipment_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "report_equipment_issue",
                "description": "Report a malfunction or issue with a medical equipment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Identifier of the medical equipment."},
                        "issue_description": {"type": "string", "description": "A description of the malfunction or issue."},
                        "severity": {"type": "string", "description": "The severity of the issue (e.g., 'critical', 'minor')."}
                    },
                    "required": ["equipment_id", "issue_description", "severity"]
                }
            }
        }
    ]
)
print(f"Medical Equipment Agent '{medical_equipment_agent.name}' created with ID: {medical_equipment_agent.id}")


# 7. Staff Scheduling Agent (Refactored from Crew Planning Agent)
staff_scheduling_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing medical staff assignments, checking availability, and optimizing schedules.",
    name="staff-scheduling-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_staff_availability",
                "description": "Retrieve the availability of medical staff (e.g., doctors, nurses) for a specific date or shift.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "staff_type": {"type": "string", "description": "Type of staff (e.g., 'doctor', 'nurse')."},
                        "date": {"type": "string", "description": "Date for availability check in YYYY-MM-DD format."}
                    },
                    "required": ["staff_type", "date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_staff_to_shift",
                "description": "Assign specific staff members to a medical shift or patient. Patient ID optional.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "staff_ids": {"type": "array", "items": {"type": "string"}, "description": "List of staff member IDs to assign."},
                        "shift_details": {"type": "string", "description": "Description of the shift (e.g., 'ER Night Shift')."},
                        "patient_id": {"type": "string", "description": "Optional patient ID if assigning to a specific patient."}
                    },
                    "required": ["staff_ids", "shift_details"]
                }
            }
        }
    ]
)
print(f"Staff Scheduling Agent '{staff_scheduling_agent.name}' created with ID: {staff_scheduling_agent.id}")


# 8. Patient Communication Agent (Refactored from Passenger Coordination Agent)
patient_communication_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for sending appointment reminders, post-visit instructions, or general health updates to patients.",
    name="patient-communication-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "send_appointment_reminder",
                "description": "Send an appointment reminder to a patient via their contact number.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Unique patient identifier."},
                        "appointment_time": {"type": "string", "description": "Time of the appointment (e.g., '2025-07-05 10:00 AM')."}
                    },
                    "required": ["patient_id", "appointment_time"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_health_update",
                "description": "Send a general health update or instruction to a patient.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Unique patient identifier."},
                        "message_content": {"type": "string", "description": "The content of the health update message."}
                    },
                    "required": ["patient_id", "message_content"]
                }
            }
        }
    ]
)
print(f"Patient Communication Agent '{patient_communication_agent.name}' created with ID: {patient_communication_agent.id}")


# 9. Clinic Operations Agent (Refactored from Operations Agent)
clinic_operations_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for overseeing overall clinic operations, monitoring patient flow, managing emergencies, and coordinating departments.",
    name="clinic-operations-agent",
    tools=[
        {"type": "web_search"}, # For general operational queries or external health news.
        {
            "type": "function",
            "function": {
                "name": "get_clinic_load",
                "description": "Retrieve the current patient load and wait times in different clinic departments.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "department": {"type": "string", "description": "Specific department (e.g., 'ER', 'Pediatrics') or 'all'.", "enum": ["ER", "Pediatrics", "Cardiology", "all"]},
                    },
                    "required": ["department"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "notify_emergency",
                "description": "Issue an emergency notification within the clinic, specifying the type and location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "emergency_type": {"type": "string", "description": "Type of emergency (e.g., 'code blue', 'fire alarm')."},
                        "location": {"type": "string", "description": "Location of the emergency (e.g., 'Room 305', 'ER waiting area')."}
                    },
                    "required": ["emergency_type", "location"]
                }
            }
        }
    ]
)
print(f"Clinic Operations Agent '{clinic_operations_agent.name}' created with ID: {clinic_operations_agent.id}")


print("\nAll new medical-related agents have been defined.")

# --- Mock functions for medical domain tools ---
# These mock functions provide simulated responses for the new medical agents' tools.

def register_patient(patient_name: str, dob: str, contact_number: str):
    """MOCK function to register a new patient."""
    print(f"\n[DEBUG] MOCK CALL: register_patient - Name: {patient_name}, DOB: {dob}, Contact: {contact_number}")
    return {"status": "success", "patient_id": "PAT-001", "message": f"Patient {patient_name} registered."}

def retrieve_patient_record(patient_id: str):
    """MOCK function to retrieve a patient's record."""
    print(f"\n[DEBUG] MOCK CALL: retrieve_patient_record for ID: {patient_id}")
    if patient_id == "PAT-001":
        return {"status": "success", "patient_id": "PAT-001", "name": "Alice Smith", "dob": "1985-11-20", "allergies": ["Penicillin"]}
    return {"status": "not_found", "message": "Patient record not found."}

def search_medical_literature(topic: str):
    """MOCK function to search medical literature."""
    print(f"\n[DEBUG] MOCK CALL: search_medical_literature for topic: {topic}")
    if "diabetes" in topic.lower():
        return {"status": "success", "results": ["Study on Type 2 Diabetes Management (2023)", "Insulin Therapies Review (2022)"], "message": f"Found articles on {topic}."}
    return {"status": "success", "results": [], "message": f"No specific literature found for {topic}."}

def analyze_lab_results(results: str):
    """MOCK function to analyze lab results."""
    print(f"\n[DEBUG] MOCK CALL: analyze_lab_results with: {results}")
    try:
        lab_data = json.loads(results)
        summary = "Lab results analysis:"
        flagged = {}
        if lab_data.get("glucose", 0) > 110:
            summary += " Glucose is high."
            flagged["glucose"] = "High"
        if lab_data.get("cholesterol", 0) > 200:
            summary += " Cholesterol is elevated."
            flagged["cholesterol"] = "Elevated"

        recommendations = "Consult with a physician for personalized advice."
        if flagged:
            recommendations += " Further tests may be recommended."

        return {
            "analysis_summary": summary,
            "recommendations": recommendations,
            "flagged_values": flagged
        }
    except json.JSONDecodeError:
        return {"analysis_summary": "Invalid JSON format for results.", "recommendations": "", "flagged_values": {}}

def check_drug_interaction(medication_names: list):
    """MOCK function to check drug interactions."""
    print(f"\n[DEBUG] MOCK CALL: check_drug_interaction for: {medication_names}")
    if "warfarin" in [m.lower() for m in medication_names] and "ibuprofen" in [m.lower() for m in medication_names]:
        return {"status": "warning", "interactions": [{"drugs": ["Warfarin", "Ibuprofen"], "severity": "High", "details": "Increased bleeding risk."}], "message": "Potential drug interactions found."}
    return {"status": "no_interaction", "interactions": [], "message": "No significant interactions found."}

def get_medication_dosage(medication_name: str):
    """MOCK function to retrieve medication dosage."""
    print(f"\n[DEBUG] MOCK CALL: get_medication_dosage for: {medication_name}")
    if "paracetamol" in medication_name.lower():
        return {"status": "success", "dosage": "500-1000mg every 4-6 hours, max 4000mg/day.", "message": "Dosage information for Paracetamol."}
    return {"status": "not_found", "dosage": "N/A", "message": "Dosage information not found for this medication."}

def monitor_vitals(patient_id: str, vitals_data: str):
    """MOCK function to record and evaluate vital signs."""
    print(f"\n[DEBUG] MOCK CALL: monitor_vitals for {patient_id} with data: {vitals_data}")
    try:
        vitals = json.loads(vitals_data)
        alert = ""
        if vitals.get("heart_rate", 0) > 100:
            alert += " High heart rate."
        if vitals.get("bp_systolic", 0) > 140:
            alert += " High systolic blood pressure."
        return {"status": "recorded", "patient_id": patient_id, "vitals": vitals, "alert": alert.strip() or "Vitals are within normal limits."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON for vitals data."}

def log_symptom(patient_id: str, symptom_description: str, severity: str):
    """MOCK function to log a patient's reported symptom."""
    print(f"\n[DEBUG] MOCK CALL: log_symptom for {patient_id}: {symptom_description} (Severity: {severity})")
    return {"status": "logged", "patient_id": patient_id, "symptom": symptom_description, "severity": severity, "log_id": "SYM-001"}

def check_equipment_status(equipment_id: str):
    """MOCK function to check medical equipment status."""
    print(f"\n[DEBUG] MOCK CALL: check_equipment_status for {equipment_id}")
    if equipment_id.upper() == "MRI-001":
        return {"status": "operational", "next_calibration": "2025-09-01", "details": "Routine check passed."}
    return {"status": "offline", "next_calibration": "N/A", "details": "Equipment not found or offline."}

def report_equipment_issue(equipment_id: str, issue_description: str, severity: str):
    """MOCK function to report an equipment issue."""
    print(f"\n[DEBUG] MOCK CALL: report_equipment_issue for {equipment_id}: {issue_description} (Severity: {severity})")
    return {"status": "issue_reported", "issue_ticket_id": "EQP-54321", "equipment": equipment_id}

def get_staff_availability(staff_type: str, date: str):
    """MOCK function to get staff availability."""
    print(f"\n[DEBUG] MOCK CALL: get_staff_availability for {staff_type} on {date}")
    if staff_type.lower() == "doctor" and date == "2025-07-01":
        return {"status": "success", "available_staff_ids": ["DR-SMITH", "DR-JONES"]}
    return {"status": "success", "available_staff_ids": []}

def assign_staff_to_shift(staff_ids: list, shift_details: str, patient_id: str = None):
    """MOCK function to assign staff to a shift."""
    print(f"\n[DEBUG] MOCK CALL: assign_staff_to_shift {staff_ids} to '{shift_details}', Patient: {patient_id}")
    return {"status": "success", "message": f"Staff {staff_ids} assigned to '{shift_details}'."}

def send_appointment_reminder(patient_id: str, appointment_time: str):
    """MOCK function to send an appointment reminder."""
    print(f"\n[DEBUG] MOCK CALL: send_appointment_reminder to {patient_id} for {appointment_time}")
    return {"status": "sent", "patient_id": patient_id, "reminder_time": appointment_time}

def send_health_update(patient_id: str, message_content: str):
    """MOCK function to send a health update."""
    print(f"\n[DEBUG] MOCK CALL: send_health_update to {patient_id} with content: '{message_content}'")
    return {"status": "sent", "patient_id": patient_id, "message": message_content}

def get_clinic_load(department: str):
    """MOCK function to get clinic load."""
    print(f"\n[DEBUG] MOCK CALL: get_clinic_load for department: {department}")
    if department.lower() == "er":
        return {"status": "success", "department": "ER", "patient_count": 15, "wait_time_minutes": 90}
    elif department.lower() == "pediatrics":
        return {"status": "success", "department": "Pediatrics", "patient_count": 5, "wait_time_minutes": 15}
    return {"status": "success", "department": department, "patient_count": 0, "wait_time_minutes": 0}

def notify_emergency(emergency_type: str, location: str):
    """MOCK function to issue an emergency notification."""
    print(f"\n[DEBUG] MOCK CALL: notify_emergency - Type: {emergency_type}, Location: {location}")
    return {"status": "notification_issued", "emergency_type": emergency_type, "location": location, "timestamp": time.time()}


# --- Master Tool Executor Mapping (Updated for Medical Domain) ---
tool_executor = {
    "register_patient": register_patient,
    "retrieve_patient_record": retrieve_patient_record,
    "search_medical_literature": search_medical_literature,
    "analyze_lab_results": analyze_lab_results,
    "check_drug_interaction": check_drug_interaction,
    "get_medication_dosage": get_medication_dosage,
    "monitor_vitals": monitor_vitals,
    "log_symptom": log_symptom,
    "check_equipment_status": check_equipment_status,
    "report_equipment_issue": report_equipment_issue,
    "get_staff_availability": get_staff_availability,
    "assign_staff_to_shift": assign_staff_to_shift,
    "send_appointment_reminder": send_appointment_reminder,
    "send_health_update": send_health_update,
    "get_clinic_load": get_clinic_load,
    "notify_emergency": notify_emergency,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: Medical information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Medical Agents ---

print("\n--- Executing Test Cases for Medical Agents (via chat completions) ---")

test_cases = [
    {
        "agent": patient_intake_agent,
        "name": "Patient Intake Agent",
        "query": "Register a new patient named Jane Doe, born on 1990-05-15, contact number 555-123-4567.",
        "expected_tool_call": "register_patient"
    },
    {
        "agent": patient_intake_agent,
        "name": "Patient Intake Agent",
        "query": "What's the record for patient ID PAT-001?",
        "expected_tool_call": "retrieve_patient_record"
    },
    {
        "agent": medical_research_agent,
        "name": "Medical Research Agent",
        "query": "Find recent studies on diabetes management.",
        "expected_tool_call": "search_medical_literature"
    },
    {
        "agent": lab_result_analyzer_agent,
        "name": "Lab Result Analyzer Agent",
        "query": "Analyze these blood test results: {\"glucose\": 130, \"cholesterol\": 260, \"hdl\": 45}",
        "expected_tool_call": "analyze_lab_results"
    },
    {
        "agent": medication_management_agent,
        "name": "Medication Management Agent",
        "query": "Are there any interactions between Warfarin and Ibuprofen?",
        "expected_tool_call": "check_drug_interaction"
    },
    {
        "agent": medication_management_agent,
        "name": "Medication Management Agent",
        "query": "What's the recommended dosage for Paracetamol?",
        "expected_tool_call": "get_medication_dosage"
    },
    {
        "agent": health_monitor_agent,
        "name": "Health Monitor Agent",
        "query": "Log vitals for patient PAT-001: heart rate 105, BP 150/90, temp 37.5C.",
        "expected_tool_call": "monitor_vitals"
    },
    {
        "agent": health_monitor_agent,
        "name": "Health Monitor Agent",
        "query": "Patient PAT-001 reports severe headache.",
        "expected_tool_call": "log_symptom"
    },
    {
        "agent": medical_equipment_agent,
        "name": "Medical Equipment Agent",
        "query": "Check status of MRI-001.",
        "expected_tool_call": "check_equipment_status"
    },
    {
        "agent": medical_equipment_agent,
        "name": "Medical Equipment Agent",
        "query": "Report minor issue with MRI-001: image distortion.",
        "expected_tool_call": "report_equipment_issue"
    },
    {
        "agent": staff_scheduling_agent,
        "name": "Staff Scheduling Agent",
        "query": "Are there any doctors available on July 1st, 2025?",
        "expected_tool_call": "get_staff_availability"
    },
    {
        "agent": staff_scheduling_agent,
        "name": "Staff Scheduling Agent",
        "query": "Assign DR-SMITH and NURSE-LISA to ER Night Shift.",
        "expected_tool_call": "assign_staff_to_shift"
    },
    {
        "agent": patient_communication_agent,
        "name": "Patient Communication Agent",
        "query": "Send appointment reminder to PAT-001 for 2025-07-05 10:00 AM.",
        "expected_tool_call": "send_appointment_reminder"
    },
    {
        "agent": patient_communication_agent,
        "name": "Patient Communication Agent",
        "query": "Send health update to PAT-001: 'Please remember to take your medication daily.'",
        "expected_tool_call": "send_health_update"
    },
    {
        "agent": clinic_operations_agent,
        "name": "Clinic Operations Agent",
        "query": "What's the current patient load in the ER?",
        "expected_tool_call": "get_clinic_load"
    },
    {
        "agent": clinic_operations_agent,
        "name": "Clinic Operations Agent",
        "query": "Notify Code Blue in Room 305.",
        "expected_tool_call": "notify_emergency"
    },
    {
        "agent": medical_research_agent, # Reusing for web search example
        "name": "Medical Research Agent (Web Search Example)",
        "query": "Latest guidelines for treating hypertension.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")



Mounted at /content/gdrive
Creating AI agents for Medical Domain...
Patient Intake Agent 'patient-intake-agent' created with ID: ag_0685697fef117f368000e8bc3ed24bcc
Medical Research Agent 'medical-research-agent' created with ID: ag_0685697ff38f7376800003cc23354396
Lab Result Analyzer Agent 'lab-result-analyzer-agent' created with ID: ag_0685697ff66c71a18000f7b77405c85a
Medication Management Agent 'medication-management-agent' created with ID: ag_0685697ff9a57ce48000857e95536bbc
Health Monitor Agent 'health-monitor-agent' created with ID: ag_0685697ffc76746f80002d89cbd425dc
Medical Equipment Agent 'medical-equipment-agent' created with ID: ag_0685697fff8377f7800049f2132dbb1b
Staff Scheduling Agent 'staff-scheduling-agent' created with ID: ag_06856980025c72758000120331916d7f
Patient Communication Agent 'patient-communication-agent' created with ID: ag_0685698005637edb8000c6c41c9d861f
Clinic Operations Agent 'clinic-operations-agent' created with ID: ag_06856980088374f280004626b7fceac3

